In [1]:
import os
from os import listdir
from os.path import isfile, join, splitext
import sys
import csv
import re

import numpy as np
import pandas as pd
import geopandas as gpd

import usaddress

# Parsing with `usaddress` module

## Edge Cases

In [3]:
# Queens, NY address with dashes in street address
queens_ny_address = "89-22 197th St Unit 2, Hollis, NY 11423"
usaddress.tag(queens_ny_address)

(OrderedDict([('AddressNumber', '89-22'),
              ('StreetName', '197th'),
              ('StreetNamePostType', 'St'),
              ('OccupancyType', 'Unit'),
              ('OccupancyIdentifier', '2'),
              ('PlaceName', 'Hollis'),
              ('StateName', 'NY'),
              ('ZipCode', '11423')]),
 'Street Address')

In [4]:
# Southwest address with Spanish names
NM_address = "220 Camino Tres SW, Albuquerque, NM 87105"
usaddress.tag(NM_address)

(OrderedDict([('AddressNumber', '220'),
              ('StreetNamePreType', 'Camino'),
              ('StreetName', 'Tres'),
              ('StreetNamePostDirectional', 'SW'),
              ('PlaceName', 'Albuquerque'),
              ('StateName', 'NM'),
              ('ZipCode', '87105')]),
 'Street Address')

In [5]:
# grid system with no street type
UT_address = "921 3385 S, Millcreek, UT 84106"
usaddress.tag(UT_address)

(OrderedDict([('AddressNumber', '921'),
              ('StreetName', '3385'),
              ('StreetNamePostDirectional', 'S'),
              ('PlaceName', 'Millcreek'),
              ('StateName', 'UT'),
              ('ZipCode', '84106')]),
 'Street Address')

# Simulated Residential History Data

In [2]:
os.chdir('..')
abs_path = os.getcwd()
print(abs_path)

C:\Users\bchan\OneDrive - UW\CLAD\CLAD_Geospatial


In [3]:
# link = os.path.join(abs_path, 'data', 'test_address_20k_v1.csv')
link = os.path.join(abs_path, 'output', 'sample_spatial_join_OMOP_clean.csv')

temp = pd.read_csv(link)
temp

C:\Users\bchan\AppData\Local\Temp\ipykernel_1740\1893463588.py:4: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(link)


,Full_Addre,Place_type,source_lon,source_lat,geometry,index_right,OBJECTID,GEOID_1,NAME_1,NAMELSAD_1,...,Shape_Leng,Shape_Le_1,Shape_Area,Tribal,address_1,address_2,city,state,zip,STATE
0,"523 E BROADWAY, SOUTH BOSTON, MA 02127",AllPlacesOfWorship,-71.043522,42.335472,POINT (-71.04352199999346 42.33547200002667),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,523 E BROADWAY,NaN,SOUTH BOSTON,MA,2127,MA
1,"454 ESSEX ST, LAWRENCE, MA 01840",AllPlacesOfWorship,-71.164940,42.706213,POINT (-71.16493999969576 42.70621300000711),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,454 ESSEX ST,NaN,LAWRENCE,MA,1840,MA
2,"569 BROADWAY, NEWARK, NJ 07104",AllPlacesOfWorship,-74.162821,40.769935,POINT (-74.16282099988595 40.76993499987205),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,569 BROADWAY,NaN,NEWARK,NJ,7104,NJ
3,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",AllPlacesOfWorship,-78.747816,42.798535,POINT (-78.74781599997843 42.79853499984358),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,3210 SOUTHWESTERN BLVD,NaN,ORCHARD PARK,NY,14127,NY
4,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",AllPlacesOfWorship,-70.113777,44.428610,POINT (-70.11377699970232 44.42860999977688),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,431 CAMPGROUND RD,NaN,LIVERMORE FLS,ME,4254,ME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104097,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",UrgentCareFacs,-84.095174,34.041727,POINT (-84.09517421962246 34.04172694900986),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,1300 PEACHTREE INDUSTRIAL BOULEVARD,NaN,SUWANEE,GA,30024,GA
104098,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",UrgentCareFacs,-84.101318,33.966797,POINT (-84.10131848955332 33.96679709299809),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,2660 SATELLITE BOULEVARD NORTHWEST,NaN,DULUTH,GA,30096,GA
104099,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",UrgentCareFacs,-83.902215,34.068832,POINT (-83.90221543184028 34.06883234729177),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,3685 BRASELTON HIGHWAY,NaN,DACULA,GA,30019,GA
104100,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",UrgentCareFacs,-84.494106,37.996508,POINT (-84.49410603994914 37.99650802456039),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,1055 DOVE RUN ROAD,NaN,LEXINGTON,KY,40502,KY


In [4]:
Pub28_usaddress_template = {
   'Recipient': 'recipient',
   'AddressNumber': 'address1',
   'AddressNumberPrefix': 'address1',
   'AddressNumberSuffix': 'address1',
   'StreetName': 'address1',
   'StreetNamePreDirectional': 'address1',
   'StreetNamePreModifier': 'address1',
   'StreetNamePreType': 'address1',
   'StreetNamePostDirectional': 'address1',
   'StreetNamePostModifier': 'address1',
   'StreetNamePostType': 'address1',
   'CornerOf': 'address1',
   'IntersectionSeparator': 'address1',
   'LandmarkName': 'address1',
   'USPSBoxGroupID': 'address1',
   'USPSBoxGroupType': 'address1',
   'USPSBoxID': 'address1',
   'USPSBoxType': 'address1',
   'BuildingName': 'address2',
   'OccupancyType': 'address2',
   'OccupancyIdentifier': 'address2',
   'SubaddressIdentifier': 'address2',
   'SubaddressType': 'address2',
   'PlaceName': 'city',
   'StateName': 'state',
   'ZipCode': 'zip_code',
}

In [5]:
# 1) identify unique address strings for location records
temp_drop = temp.drop_duplicates(subset='Full_Addre')

# 2) set up OMOP placeholder table
OMOP_location = pd.DataFrame(columns=['Location_id','address_1','address_2','city','state','zip','county',
                                      'location_source_value','latitude','longitude'])
                             
# need to change datatype for latitude and longitude
OMOP_location

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude


In [6]:
# 3) assign address_strings to OMOP_location table location_source_value series
OMOP_location['location_source_value'] = temp_drop.Full_Addre
OMOP_location['Location_id'] = OMOP_location.index+1

# OMOP_location['Location_id'] = OMOP_location.re+1
OMOP_location.latitude = temp_drop.source_lat
OMOP_location.longitude = temp_drop.source_lon
OMOP_location

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude
0,1,NaN,NaN,NaN,NaN,NaN,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522
1,2,NaN,NaN,NaN,NaN,NaN,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940
2,3,NaN,NaN,NaN,NaN,NaN,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821
3,4,NaN,NaN,NaN,NaN,NaN,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816
4,5,NaN,NaN,NaN,NaN,NaN,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777
...,...,...,...,...,...,...,...,...,...,...
104097,104098,NaN,NaN,NaN,NaN,NaN,NaN,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174
104098,104099,NaN,NaN,NaN,NaN,NaN,NaN,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318
104099,104100,NaN,NaN,NaN,NaN,NaN,NaN,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215
104100,104101,NaN,NaN,NaN,NaN,NaN,NaN,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106


## Parse OMOP Components
  * address_1
  * address_2
  * city
  * state
  * zip
  * county
  * location_source_value
  * latitude
  * longitude

In [7]:
%%time
# 4) process address string through usaddress parser with Pub28 template
repo = pd.DataFrame()

# iter
for ind, each in OMOP_location.loc[:,['location_source_value']].drop_duplicates().iterrows():

    # try Pub28 parsing
    try:
        obj = usaddress.tag(each.location_source_value, tag_mapping=Pub28_usaddress_template)
        
        # staging
        tmp = pd.DataFrame(obj[0], columns=obj[0].keys(), index=[ind])
        tmp['Address_type'] = obj[1]
        
        # development
        OMOP_location.loc[ind, 'address_1'] = tmp['address1'].values[0]
        OMOP_location.loc[ind, 'city'] = tmp['city'].values[0]
        OMOP_location.loc[ind, 'state'] = tmp['state'].values[0]
        OMOP_location.loc[ind, 'zip'] = tmp['zip_code'].values[0]
        OMOP_location.loc[ind, 'address_type']=tmp['Address_type'].values[0]

        address_2 = tmp['address2'].values[0]
        if len(address_2) >= 3:
            OMOP_location.loc[ind, 'address_2'] = address_2
        else:
            OMOP_location.loc[ind, 'address_2'] = np.NaN

        repo = repo.append(tmp)
    
    except:
        # print(ind, each.location_source_value)
        pass
   
    # if ind==15:
    #     break

CPU times: total: 6min 11s
Wall time: 10min 49s


In [8]:
OMOP_location

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type
0,1,523 E BROADWAY,NaN,SOUTH BOSTON,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address
1,2,454 ESSEX ST,NaN,LAWRENCE,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address
2,3,569 BROADWAY,NaN,NEWARK,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address
3,4,3210 SOUTHWESTERN BLVD,NaN,ORCHARD PARK,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address
4,5,431 CAMPGROUND RD,NaN,LIVERMORE FLS,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address
...,...,...,...,...,...,...,...,...,...,...,...
104097,104098,1300 PEACHTREE INDUSTRIAL BOULEVARD,NaN,SUWANEE,GA,30024,NaN,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address
104098,104099,2660 SATELLITE BOULEVARD NORTHWEST,NaN,DULUTH,GA,30096,NaN,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address
104099,104100,3685 BRASELTON HIGHWAY,NaN,DACULA,GA,30019,NaN,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address
104100,104101,1055 DOVE RUN ROAD,NaN,LEXINGTON,KY,40502,NaN,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address


In [9]:
OMOP_location.loc[OMOP_location.address_2.notna()]

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type
6,7,337 STATE STREET,SUITE 3,AUGUSTA,ME,04330,NaN,"337 STATE STREET SUITE 3, AUGUSTA, ME 04330",44.297293,-69.783541,Street Address
79,80,75 MORTON VILLAGE DR,APT 408,MATTAPAN,MA,02126,NaN,"75 MORTON VILLAGE DR APT 408, MATTAPAN, MA 02126",42.280395,-71.085149,Street Address
81,82,200 HANCOCK ST,APT 904,BANGOR,ME,04401,NaN,"200 HANCOCK ST APT 904, BANGOR, ME 04401",44.802406,-68.762050,Street Address
130,131,430 GRANDVIEW AVE,APT 17,BANGOR,ME,04401,NaN,"430 GRANDVIEW AVE APT 17, BANGOR, ME 04401",44.827785,-68.781941,Street Address
187,188,33 GLENWOOD DR,APT 2,BANGOR,ME,04401,NaN,"33 GLENWOOD DR APT 2, BANGOR, ME 04401",44.823394,-68.795611,Street Address
...,...,...,...,...,...,...,...,...,...,...,...
100989,100990,1998 UNITED STATES HIGHWAY,62 412,HIGHLAND,AR,72542,NaN,"1998 UNITED STATES HIGHWAY 62 412, HIGHLAND, A...",36.261774,-91.526763,Street Address
101221,101222,19422 UNITED STATES HIGHWAY,281,"NORTH, SAN ANTONIO",TX,78258,NaN,"19422 UNITED STATES HIGHWAY 281 NORTH, SAN ANT...",29.621960,-98.463141,Street Address
102819,102820,9511 UNITED STATES HIGHWAY,431,ALBERTVILLE,AL,35950,NaN,"9511 UNITED STATES HIGHWAY 431, ALBERTVILLE, A...",34.288373,-86.241246,Street Address
103263,103264,10935 SOUTH UNITED STATES HIGHWAY,15 501,SOUTHERN PINES,NC,28387,NaN,"10935 SOUTH UNITED STATES HIGHWAY 15 501, SOUT...",35.162689,-79.420724,Street Address


# Post-Hoc Processing

In [10]:
# replace state full name to abbreviation
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

def multipleReplace(text, wordDict):
    for key in wordDict:
        text = text.replace(key, wordDict[key])
    return text

In [11]:
# replace full state names to abbreviations
OMOP_location['state_abbr'] = OMOP_location.state.apply(lambda x: multipleReplace(str(x).strip(), us_state_to_abbrev))
OMOP_location

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
0,1,523 E BROADWAY,NaN,SOUTH BOSTON,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA
1,2,454 ESSEX ST,NaN,LAWRENCE,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA
2,3,569 BROADWAY,NaN,NEWARK,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ
3,4,3210 SOUTHWESTERN BLVD,NaN,ORCHARD PARK,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY
4,5,431 CAMPGROUND RD,NaN,LIVERMORE FLS,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME
...,...,...,...,...,...,...,...,...,...,...,...,...
104097,104098,1300 PEACHTREE INDUSTRIAL BOULEVARD,NaN,SUWANEE,GA,30024,NaN,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA
104098,104099,2660 SATELLITE BOULEVARD NORTHWEST,NaN,DULUTH,GA,30096,NaN,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA
104099,104100,3685 BRASELTON HIGHWAY,NaN,DACULA,GA,30019,NaN,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA
104100,104101,1055 DOVE RUN ROAD,NaN,LEXINGTON,KY,40502,NaN,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY


In [12]:
# capitalize only first letter
OMOP_location['address_1'] = OMOP_location.address_1.apply(lambda x: str(x).strip().title())
OMOP_location['address_2'] = OMOP_location.address_2.apply(lambda x: str(x).strip().title() if not np.NaN else x)
OMOP_location['city'] = OMOP_location.city.apply(lambda x: str(x).strip().title())

OMOP_location

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
0,1,523 E Broadway,NaN,South Boston,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA
1,2,454 Essex St,NaN,Lawrence,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA
2,3,569 Broadway,NaN,Newark,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ
3,4,3210 Southwestern Blvd,NaN,Orchard Park,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY
4,5,431 Campground Rd,NaN,Livermore Fls,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME
...,...,...,...,...,...,...,...,...,...,...,...,...
104097,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,GA,30024,NaN,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA
104098,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,GA,30096,NaN,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA
104099,104100,3685 Braselton Highway,NaN,Dacula,GA,30019,NaN,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA
104100,104101,1055 Dove Run Road,NaN,Lexington,KY,40502,NaN,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY


In [14]:
OMOP_location_path = os.path.join(abs_path, 'output', 'OMOP_location.csv')

OMOP_location.to_csv(OMOP_location_path, index=False)

# Data Quality Check
* address_1 should not be null and must be alphanumeric combination row-wise, may contain special characters
* address_2 can be null, otherwise alphanumeric combination
* city cannot have non-alphanumeric characters
* state must be 2-character abbreviation
* filter out non-street addresses by `address_type`

1) Parse with `usaddress` library
2) Data quality check above
3) Run custom parser function

* TO DO
  * Fix `West Virginia` state abbreviation

## Parse Failed Addresses with Custom Parser

In [40]:
OMOP_location_path = pd.read_csv(os.path.join(abs_path, 'output', 'OMOP_location.csv'))

In [14]:
def parse_address(df,
                  address_col,
                  state_full_pattern,
                  state_abbr_pattern,
                  zip_code_pattern=r"[0-9]{5}(?:-[0-9]{4})?"
                  # state_code_pattern=r"[A-Z][a-z]+(?: +[A-Z][a-z]+)*)"
                 ):
    """
    Parse full address string to OMOP components by Regex search

    Parameters
    ----------
    df (DataFrame): Pandas DataFrame of source centroids with dates and user IDs

    Returns
    -------
    parse_df (DataFrame): DataFrame with parsed OMOP address components
    """
    tmp = []
    for i,row in df.iterrows():
        addr_components = row[address_col].split(',')

        # parse address if no RegEx match for 'APT'
        if len(re.findall(r'APT', row[address_col], flags=re.IGNORECASE)) == 0:
            state_zip = addr_components[2].split(' ')
            if len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0:
                row['address_1'] = addr_components[0]
                row['address_2'] = np.NaN
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)   
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                row['address_1'] = addr_components[0]
                row['address_2'] = np.NaN
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = np.NaN
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0 and len(re.findall(state_full_pattern, addr_components[-1].title())) > 0:
                row['address_1'] = addr_components[0]
                row['address_2'] = np.NaN
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                row['address_1'] = addr_components[0]
                row['address_2'] = np.NaN
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())
                row['zip'] = np.NaN
    
                tmp.append(row)
                
        # # parse address if RegEx match for 'APT' to address_1 & address_2
        elif len(re.findall(r'APT', row[address_col], flags=re.IGNORECASE)) > 0:
            if len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0:
                base_address = addr_components[0]
                apt_string = re.findall(r'APT', row.Full_Address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(apt_string)[0]
                row['address_2'] = base_address.partition(apt_string)[1] + base_address.partition(apt_string)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                base_address = addr_components[0]
                apt_string = re.findall(r'APT', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(apt_string)[0]
                row['address_2'] = base_address.partition(apt_string)[1] + base_address.partition(apt_string)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = np.NaN
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0 and len(re.findall(state_full_pattern, addr_components[-1].title())) > 0:
                base_address = addr_components[0]
                apt_string = re.findall(r'APT', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(apt_string)[0]
                row['address_2'] = base_address.partition(apt_string)[1] + base_address.partition(apt_string)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                base_address = addr_components[0]
                apt_string = re.findall(r'APT', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(apt_string)[0]
                row['address_2'] = base_address.partition(apt_string)[1] + base_address.partition(apt_string)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())
                row['zip'] = np.NaN
    
                tmp.append(row)
                    
        # # parse address if RegEx match for 'Suite' to to address_1 & address_2
        elif len(re.findall(r'SUITE', row[address_col], flags=re.IGNORECASE)) > 0:
            if len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0:
                base_address = addr_components[0]
                suite_str = re.findall(r'SUITE', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(suite_str)[0]
                row['address_2'] = base_address.partition(suite_str)[1] + base_address.partition(suite_str)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]

                tmp.append(row)

            elif len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                base_address = addr_components[0]
                suite_str = re.findall(r'SUITE', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(suite_str)[0]
                row['address_2'] = base_address.partition(suite_str)[1] + base_address.partition(suite_str)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = np.NaN
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0 and len(re.findall(state_full_pattern, addr_components[-1].title())) > 0:
                base_address = addr_components[0]
                suite_str = re.findall(r'SUITE', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(suite_str)[0]
                row['address_2'] = base_address.partition(suite_str)[1] + base_address.partition(suite_str)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                base_address = addr_components[0]
                suite_str = re.findall(r'SUITE', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(suite_str)[0]
                row['address_2'] = base_address.partition(suite_str)[1] + base_address.partition(suite_str)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())
                row['zip'] = np.NaN
    
                tmp.append(row)        
        
    return pd.DataFrame(tmp)

In [15]:
# get failed addresses that do not have correct state abbreviation
OMOP_location['state_abbr'] = OMOP_location['state_abbr'].astype(str)
OMOP_state_failed = OMOP_location.loc[OMOP_location.state_abbr.str.len() > 2]
# state_condition = np.where(OMOP_location['state_abbr'].str.len() > 2)
# OMOP_state_failed = OMOP_location.loc[OMOP_location.state_abbr.map(lambda x: ]
OMOP_state_failed

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
221,222,Nan,NaN,Nan,NaN,NaN,NaN,"LOCAL, KINGMAN, ME 04451",45.639023,-68.263641,nan,nan
561,562,Nan,NaN,Nan,NaN,NaN,NaN,"LOCAL, KINGFIELD, ME 04947",44.960341,-70.159052,nan,nan
762,763,Nan,NaN,Nan,NaN,NaN,NaN,"LOCAL, BROOKLINE, NH 03033",42.736111,-71.663431,nan,nan
1073,1074,Nan,NaN,Nan,NaN,NaN,NaN,"LOCAL, WONALANCET, NH 03897",43.908447,-71.344152,nan,nan
1231,1232,Nan,NaN,Nan,NaN,NaN,NaN,"GENERAL DELIVERY, N HARTLAND, VT 05052",43.591020,-72.356611,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...
95864,95865,Nan,NaN,Nan,NaN,NaN,NaN,"25 MI N OF GALLUP NM HWY 491, TOHATCHI, NM 87325",35.848832,-108.745451,nan,nan
96830,96831,Nan,NaN,Nan,NaN,NaN,NaN,"BLDG 25000TH KAFB EAST, ALBUQUERQUE, NM 87116",35.058916,-106.555920,nan,nan
98094,98095,Nan,NaN,Nan,NaN,NaN,NaN,"DEPT 3374, LARAMIE, WY 82071",41.313964,-105.584531,nan,nan
98667,98668,Nan,NaN,Nan,NaN,NaN,NaN,COLLEGE OF SOUTHERN IDAHO EVERGREEN BLDG. ROOM...,42.579849,-114.473937,nan,nan


In [16]:
%%time
# run custom parser on failed addresses above

state_full_pattern = r"/AL|Alabama|AK|Alaska|AZ|Arizona|AR|Arkansas|CA|California|CO|Colorado|CT|Connecticut|DE|Delaware|FL|Florida|GA|Georgia|HI|Hawaii|ID|Idaho|IL|Illinois|IN|Indiana|IA|Iowa|KS|Kansas|KY|Kentucky|LA|Louisiana|ME|Maine|MD|Maryland|MA|Massachusetts|MI|Michigan|MN|Minnesota|MS|Mississippi|MO|Missouri|MT|Montana|NE|Nebraska|NV|Nevada|NH|New Hampshire|NJ|New Jersey|NM|New Mexico|NY|New York|NC|North Carolina|ND|North Dakota|OH|Ohio|OK|Oklahoma|OR|Oregon|PA|Pennsylvania|RI|Rhode Island|SC|South Carolina|SD|South Dakota|TN|Tennessee|TX|Texas|UT|Utah|VT|Vermont|VA|Virginia|WA|Washington|WV|West Virginia|WI|Wisconsin|WY|Wyoming/"
state_abbr_pattern = re.compile(r'\b(AZ|CA|...|NJ|N\.J\.|NM|N\.M\.|...)\b')
state_abbr_case = r"^([Aa][LKSZRAEPlkszraep]|[Cc][AOTaot]|[Dd][ECec]|[Ff][LMlm]|[Gg][AUau]|[Hh][Ii]|[Ii][ADLNadln]|[Kk][SYsy]|[Ll][Aa]|[Mm][ADEHINOPSTadehinopst]|[Nn][CDEHJMVYcdehjmvy]|[Oo][HKRhkr]|[Pp][ARWarw]|[Rr][Ii]|[Ss][CDcd]|[Tt][NXnx]|[Uu][Tt]|[Vv][AITait]|[Ww][AIVYaivy])$"
zip_code_pattern=r"[0-9]{5}(?:-[0-9]{4})?"

new_df = parse_address(df=OMOP_state_failed,
                       address_col = 'location_source_value',
                       state_full_pattern=state_full_pattern, 
                       state_abbr_pattern=state_abbr_pattern)

new_df

CPU times: total: 297 ms
Wall time: 1.27 s


,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
221,222,LOCAL,NaN,KINGMAN,ME,04451,NaN,"LOCAL, KINGMAN, ME 04451",45.639023,-68.263641,nan,nan
561,562,LOCAL,NaN,KINGFIELD,ME,04947,NaN,"LOCAL, KINGFIELD, ME 04947",44.960341,-70.159052,nan,nan
762,763,LOCAL,NaN,BROOKLINE,NH,03033,NaN,"LOCAL, BROOKLINE, NH 03033",42.736111,-71.663431,nan,nan
1073,1074,LOCAL,NaN,WONALANCET,NH,03897,NaN,"LOCAL, WONALANCET, NH 03897",43.908447,-71.344152,nan,nan
1231,1232,GENERAL DELIVERY,NaN,N HARTLAND,VT,05052,NaN,"GENERAL DELIVERY, N HARTLAND, VT 05052",43.591020,-72.356611,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...
95864,95865,25 MI N OF GALLUP NM HWY 491,NaN,TOHATCHI,NM,87325,NaN,"25 MI N OF GALLUP NM HWY 491, TOHATCHI, NM 87325",35.848832,-108.745451,nan,nan
96830,96831,BLDG 25000TH KAFB EAST,NaN,ALBUQUERQUE,NM,87116,NaN,"BLDG 25000TH KAFB EAST, ALBUQUERQUE, NM 87116",35.058916,-106.555920,nan,nan
98094,98095,DEPT 3374,NaN,LARAMIE,WY,82071,NaN,"DEPT 3374, LARAMIE, WY 82071",41.313964,-105.584531,nan,nan
98667,98668,COLLEGE OF SOUTHERN IDAHO EVERGREEN BLDG. ROOM 38,NaN,TWIN FALLS,ID,83301,NaN,COLLEGE OF SOUTHERN IDAHO EVERGREEN BLDG. ROOM...,42.579849,-114.473937,nan,nan


In [17]:
new_df['state_abbr'] = new_df.state.apply(lambda x: multipleReplace(str(x).strip(), us_state_to_abbrev))
new_df

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
221,222,LOCAL,NaN,KINGMAN,ME,04451,NaN,"LOCAL, KINGMAN, ME 04451",45.639023,-68.263641,nan,ME
561,562,LOCAL,NaN,KINGFIELD,ME,04947,NaN,"LOCAL, KINGFIELD, ME 04947",44.960341,-70.159052,nan,ME
762,763,LOCAL,NaN,BROOKLINE,NH,03033,NaN,"LOCAL, BROOKLINE, NH 03033",42.736111,-71.663431,nan,NH
1073,1074,LOCAL,NaN,WONALANCET,NH,03897,NaN,"LOCAL, WONALANCET, NH 03897",43.908447,-71.344152,nan,NH
1231,1232,GENERAL DELIVERY,NaN,N HARTLAND,VT,05052,NaN,"GENERAL DELIVERY, N HARTLAND, VT 05052",43.591020,-72.356611,nan,VT
...,...,...,...,...,...,...,...,...,...,...,...,...
95864,95865,25 MI N OF GALLUP NM HWY 491,NaN,TOHATCHI,NM,87325,NaN,"25 MI N OF GALLUP NM HWY 491, TOHATCHI, NM 87325",35.848832,-108.745451,nan,NM
96830,96831,BLDG 25000TH KAFB EAST,NaN,ALBUQUERQUE,NM,87116,NaN,"BLDG 25000TH KAFB EAST, ALBUQUERQUE, NM 87116",35.058916,-106.555920,nan,NM
98094,98095,DEPT 3374,NaN,LARAMIE,WY,82071,NaN,"DEPT 3374, LARAMIE, WY 82071",41.313964,-105.584531,nan,WY
98667,98668,COLLEGE OF SOUTHERN IDAHO EVERGREEN BLDG. ROOM 38,NaN,TWIN FALLS,ID,83301,NaN,COLLEGE OF SOUTHERN IDAHO EVERGREEN BLDG. ROOM...,42.579849,-114.473937,nan,ID


In [18]:
# update OMOP_location with addresses parsed with custom parser
OMOP_location_updated = new_df.combine_first(OMOP_location)
OMOP_location_updated

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr
0,1,523 E Broadway,NaN,South Boston,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA
1,2,454 Essex St,NaN,Lawrence,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA
2,3,569 Broadway,NaN,Newark,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ
3,4,3210 Southwestern Blvd,NaN,Orchard Park,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY
4,5,431 Campground Rd,NaN,Livermore Fls,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME
...,...,...,...,...,...,...,...,...,...,...,...,...
104097,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,GA,30024,NaN,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA
104098,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,GA,30096,NaN,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA
104099,104100,3685 Braselton Highway,NaN,Dacula,GA,30019,NaN,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA
104100,104101,1055 Dove Run Road,NaN,Lexington,KY,40502,NaN,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY


## Classify Failures
* 0: SUCCESSFUL GEOCODE
* 1: FAILED GEOCODE (UNSPECIFIED)
* 2:  FAILED DUE TO PO BOX ADDRESS (CANNOT BE RECOVERED FROM FAILURE)
* 3:  FAILED GEOCODE (UNSPECIFIED) AND EXCLUDED DUE TO NON-WASHINGTON STATE AND NON-WASHINGTON ZIP CODE ENCODING
* 4:  FAILED DUE TO RETURNED / UNDELIVERABLE / HOMELESS ANNOTATION [CANNOT BE RECOVERED FROM FAILURE]
* 5:  FAILED DUE TO INCOMPLETE ADDRESS (ADDRESS LINES WERE EITHER ALL NUMBERS OR ALL LETTERS)  [CANNOT BE RECOVERED FROM FAILURE]
* 6:  FAILED BECAUSE STREET ADDRESS IN LINE1 IS FLIPPED WITH LINE 2 (e.g., LINE 1 is the apartment number and LINE 2 has street address) 
* 7:  FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS
* 8:  FAILED DUE TO ADDRESS LINES CONTAINING NAMES or MAILBOX NUMBERS

In [147]:
def custom_flag(x):
    """
    Post-hoc data quality check of parsed addresses
    """  
    # if 'PO' in str(x.address_1) or 'P.O.' in str(x.address_1):
    # if 'PO' in str(x.address_1).strip() or 'P.O.' in str(x.address_1):
    if re.match('PO', x.address_1, re.IGNORECASE):
        return 'FAILED DUE TO PO BOX ADDRESS'
    elif re.match('APT', x.address_1, re.IGNORECASE) or re.match('SUITE', x.address_1, re.IGNORECASE):
        return 'FAILED DUE TO STREET ADDRESS IN LINE_1 IS FLIPPED WITH LINE_2'
    elif not x.address_1[0].isdigit():
        return 'FAILED DUE TO STREET ADDRESS STARTS WITH LETTER'
    # check address_1 only contains alphanumeric characters (spaces are ok)
    elif any(not c.isalnum() and not c.isspace() for c in x.address_1):
        return 'FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS'
    elif len(x.state_abbr) > 2:
        return 'FAILED DUE TO INCORRECT STATE FORMAT'
    elif x[['address_1', 'city', 'state', 'zip']].isnull().any():
        return 'FAILED DUE TO INCOMPLETE PARSING'
    else:
        return 'SUCCESSFUL ADDRESS'

In [148]:
%%time
OMOP_location_updated['flag'] = OMOP_location_updated.apply(lambda x: custom_flag(x), axis=1)

OMOP_location_updated

CPU times: total: 2.77 s
Wall time: 5.16 s


,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,1,2499 South Wilmington Avenue,NaN,Compton,CA,90220,NaN,"2499 SOUTH WILMINGTON AVENUE, COMPTON, CA 90220",33.868428,-118.236303,Street Address,CA,SUCCESSFUL ADDRESS
1,2,420 South Pierre Street,NaN,Pierre,South Dakota,57501,NaN,"420 South Pierre Street, Pierre, South Dakota ...",43.254257,-98.695554,Street Address,SD,SUCCESSFUL ADDRESS
2,3,12150 30 Mile Road,NaN,Washington,MI,48095,NaN,"12150 30 MILE ROAD, WASHINGTON, MI 48095",42.771167,-83.013787,Street Address,MI,SUCCESSFUL ADDRESS
3,4,65 Tifton Eldorado Rd,NaN,Tifton,GA,31794,NaN,"65 TIFTON ELDORADO RD, TIFTON, GA 31794",31.445218,-83.496325,Street Address,GA,SUCCESSFUL ADDRESS
4,5,12512 Walters Rd,NaN,Houston,TX,77014,NaN,"12512 WALTERS RD, HOUSTON, TX 77014",29.973519,-95.468105,Street Address,TX,SUCCESSFUL ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,19995,W1828 Gros Cap Road,NaN,Saint Ignace,MI,49781,NaN,"W1828 GROS CAP ROAD, SAINT IGNACE, MI 49781",45.870412,-84.823893,Street Address,MI,FAILED DUE TO STREET ADDRESS STARTS WITH LETTER
19995,19996,730 Reems Creek Road,NaN,Weaverville,NC,28787,NaN,"730 Reems Creek Road, Weaverville, NC 28787",35.694465,-82.509238,Street Address,NC,SUCCESSFUL ADDRESS
19996,19997,5793 South 700 East,NaN,Whitestown,IN,46075,NaN,"5793 South 700 East, Whitestown, IN 46075",39.955682,-86.335915,Street Address,IN,SUCCESSFUL ADDRESS
19997,19998,7150 E Arch Rd,NaN,Stockton,CA,95205,NaN,"7150 E ARCH RD, STOCKTON, CA 95205",37.902070,-121.187052,Street Address,CA,SUCCESSFUL ADDRESS


In [150]:
OMOP_location_updated.flag.value_counts()

flag
SUCCESSFUL ADDRESS                                 12879
FAILED DUE TO STREET ADDRESS STARTS WITH LETTER      821
FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS         588
FAILED DUE TO INCORRECT STATE FORMAT                  11
FAILED DUE TO PO BOX ADDRESS                          11
FAILED DUE TO INCOMPLETE PARSING                       1
Name: count, dtype: int64

In [152]:
OMOP_location_updated_path = os.path.join(abs_path, 'output', 'OMOP_location_flagged.csv')

OMOP_location_updated.to_csv(OMOP_location_updated_path, index=False)

In [19]:
def custom_flag(x):
    """
    Post-hoc data quality check of parsed addresses
    """  
    # check if PO box address
    if 'PO' in str(x.address_1) or 'PO' in str(x.address_2)\
        or 'P.O' in str(x.address_1) or 'P.O' in str(x.address_2)\
        or 'P O' in str(x.address_1) or 'P O' in str(x.address_2)\
        or 'PSC' in str(x.address_1) or 'PSC' in str(x.address_2)\
        or 'PNB' in str(x.address_1) or 'PNB' in str(x.address_2)\
        or 'PMB' in str(x.address_1) or 'PMB' in str(x.address_2):
        return 'FAILED DUE TO PO BOX ADDRESS'
    # check if address_2 and address_1 are flipped
    elif re.match('APT', x.address_1, re.IGNORECASE) or re.match('SUITE', x.address_1, re.IGNORECASE):
        return 'FAILED DUE TO STREET ADDRESS IN LINE_1 IS FLIPPED WITH LINE_2'
    # check if street address starts with a non-digit character
    elif not x.address_1[0].isdigit():
        return 'FAILED DUE TO STREET ADDRESS STARTS WITH LETTER'
    # check address_1 only contains alphanumeric characters (spaces are ok)
    elif any(not c.isalnum() and not c.isspace() for c in x.address_1):
        return 'FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS'
    # check if parsed 'state' component matches a US state or territory abbreviation
    elif len(x.state_abbr) > 2:
        return 'FAILED DUE TO INCORRECT STATE FORMAT'
    # check if any of the required address components did not parse from the full address
    elif x[['address_1', 'city', 'state', 'zip']].isnull().any():
        return 'FAILED DUE TO INCOMPLETE PARSING'
    else:
        return 'SUCCESSFUL ADDRESS'

In [20]:
%%time
OMOP_location_updated['flag'] = OMOP_location_updated.apply(lambda x: custom_flag(x), axis=1)

OMOP_location_updated

CPU times: total: 11.2 s
Wall time: 42.4 s


,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,1,523 E Broadway,NaN,South Boston,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA,SUCCESSFUL ADDRESS
1,2,454 Essex St,NaN,Lawrence,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA,SUCCESSFUL ADDRESS
2,3,569 Broadway,NaN,Newark,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ,SUCCESSFUL ADDRESS
3,4,3210 Southwestern Blvd,NaN,Orchard Park,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS
4,5,431 Campground Rd,NaN,Livermore Fls,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME,SUCCESSFUL ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104097,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,GA,30024,NaN,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA,SUCCESSFUL ADDRESS
104098,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,GA,30096,NaN,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS
104099,104100,3685 Braselton Highway,NaN,Dacula,GA,30019,NaN,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS
104100,104101,1055 Dove Run Road,NaN,Lexington,KY,40502,NaN,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS


In [21]:
OMOP_location_updated.flag.value_counts()

flag
SUCCESSFUL ADDRESS                                               82368
FAILED DUE TO STREET ADDRESS STARTS WITH LETTER                   9007
FAILED DUE TO PRESENCE OF SPECIAL CHARACTERS                      3039
FAILED DUE TO PO BOX ADDRESS                                       226
FAILED DUE TO INCORRECT STATE FORMAT                                33
FAILED DUE TO INCOMPLETE PARSING                                     2
FAILED DUE TO STREET ADDRESS IN LINE_1 IS FLIPPED WITH LINE_2        1
Name: count, dtype: int64

In [22]:
OMOP_location_updated_success = OMOP_location_updated.loc[OMOP_location_updated.flag == 'SUCCESSFUL ADDRESS']
OMOP_location_updated_success

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,1,523 E Broadway,NaN,South Boston,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA,SUCCESSFUL ADDRESS
1,2,454 Essex St,NaN,Lawrence,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA,SUCCESSFUL ADDRESS
2,3,569 Broadway,NaN,Newark,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ,SUCCESSFUL ADDRESS
3,4,3210 Southwestern Blvd,NaN,Orchard Park,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS
4,5,431 Campground Rd,NaN,Livermore Fls,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME,SUCCESSFUL ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104097,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,GA,30024,NaN,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA,SUCCESSFUL ADDRESS
104098,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,GA,30096,NaN,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS
104099,104100,3685 Braselton Highway,NaN,Dacula,GA,30019,NaN,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS
104100,104101,1055 Dove Run Road,NaN,Lexington,KY,40502,NaN,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS


In [23]:
OMOP_location_updated_path = os.path.join(abs_path, 'output', 'OMOP_location_flagged_successful.csv')

OMOP_location_updated_success.to_csv(OMOP_location_updated_path, index=False)

In [31]:
pd.DataFrame(OMOP_location_updated_success.state_abbr.sort_values().value_counts())

,count
state_abbr,
SD,8401
CA,6526
CO,6029
AZ,5495
OR,4289
NE,3384
UT,2758
NM,2396
FL,2363
